<a href="https://colab.research.google.com/github/srinath-96/SubRedditAgent/blob/main/SimpleRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install -q google-adk google-generativeai numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.4 MB/s eta 0:00:00


In [2]:
import os
import google.generativeai as genai
from google.adk.agents import Agent
from google.adk.tools import FunctionTool
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types as adk_types
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import asyncio
import traceback

In [22]:


try:


  if 'GOOGLE_API_KEY' not in os.environ:

      from google.colab import userdata
      os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

  if not os.environ.get('GOOGLE_API_KEY'):
      print("ERROR: GOOGLE_API_KEY not found.")
      print("Please set it using Colab Secrets (recommended) or by uncommenting and")
      print("replacing 'YOUR_API_KEY' in the code above.")

  else:
      genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
      print("Google API Key configured successfully.")

except ImportError:
    print("Could not import google.colab.userdata. Are you running in Colab?")
    if not os.environ.get('GOOGLE_API_KEY'):
        print("ERROR: GOOGLE_API_KEY not found. Please set the environment variable.")

except Exception as e:
    print(f"An error occurred during API key configuration: {e}")


Google API Key configured successfully.


In [24]:
import json
from google.colab import files

print("Please upload your Reddit posts JSON file:")
uploaded = files.upload()

json_filename = None
if uploaded:
    json_filename = list(uploaded.keys())[0]
    print(f"\nUploaded file: '{json_filename}'")
else:
    print("No file uploaded. Cannot proceed with data loading.")


Please upload your Reddit posts JSON file:


Saving wallstreetbets_week_250posts_20250425_183349.json to wallstreetbets_week_250posts_20250425_183349 (1).json

Uploaded file: 'wallstreetbets_week_250posts_20250425_183349 (1).json'


In [26]:
pip install langchain_community langchain_text_splitters langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [29]:
pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 18.9 MB/s eta 0:00:00


In [31]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 23.8 MB/s eta 0:00:

In [27]:
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

In [32]:
vectorstore = None
embeddings = None
if json_filename and os.environ.get('GOOGLE_API_KEY'):
    print(f"\n--- Setting up LangChain RAG pipeline for {json_filename} ---")

    def metadata_func(record: dict, metadata: dict) -> dict:
        metadata["post_id"] = record.get("id", "N/A")
        metadata["url"] = record.get("url", "N/A")
        metadata["score"] = record.get("score", 0)

        return metadata


    loader = JSONLoader(
        file_path=json_filename,
        jq_schema='.[ ] | {content: ("Title: " + .title + "\nBody: " + (.body // "")), id: .id, url: .url, score: .score}', # Extract content and metadata directly
        content_key="content",
        metadata_func=metadata_func,
        text_content=False
    )


    try:
        print("Loading documents via LangChain JSONLoader...")
        documents = loader.load()
        print(f"Loaded {len(documents)} documents initially.")

        documents = [doc for doc in documents if doc.page_content.strip() and doc.page_content != "Title: \nBody: "]
        print(f"Filtered down to {len(documents)} documents with content.")

        if documents:
            print("\nSample Document 1 Metadata:", documents[0].metadata)



        print("\nInitializing text splitter...")
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=3000,
            chunk_overlap=200
        )


        print("Splitting documents into chunks...")
        docs_split = text_splitter.split_documents(documents)
        print(f"Split into {len(docs_split)} chunks.")

        if docs_split:

           print("\nInitializing Google Generative AI Embeddings...")

           embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # Or "models/embedding-001"


           print("Creating Chroma vector store and embedding chunks...")

           vectorstore = Chroma.from_documents(
               documents=docs_split,
               embedding=embeddings,

           )
           print("Chroma vector store created successfully.")
        else:
            print("No document chunks to process after splitting. Vector store not created.")

    except Exception as e:
        print(f"An error occurred during LangChain processing: {e}")
        traceback.print_exc()

else:
    if not json_filename:
        print("Skipping LangChain setup: JSON file not uploaded.")
    else:
        print("Skipping LangChain setup: Google API Key not configured.")


--- Setting up LangChain RAG pipeline for wallstreetbets_week_250posts_20250425_183349 (1).json ---
Loading documents via LangChain JSONLoader...
Loaded 250 documents initially.
Filtered down to 250 documents with content.

Sample Document 1 Metadata: {'source': '/content/wallstreetbets_week_250posts_20250425_183349 (1).json', 'seq_num': 1, 'post_id': '1k5da53', 'url': 'https://i.redd.it/2vaajd0mgfwe1.png', 'score': 48717}

Initializing text splitter...
Splitting documents into chunks...
Split into 282 chunks.

Initializing Google Generative AI Embeddings...
Creating Chroma vector store and embedding chunks...
Chroma vector store created successfully.


In [34]:


def retrieve_relevant_context_langchain(query: str) -> dict:
    """
    Retrieves relevant document chunks from the LangChain vector store
    based on semantic similarity to the user's query. This tool searches
    an indexed knowledge base of Reddit posts.
    """
    print(f"\n--- LangChain Retrieval Function called with query: '{query}' ---")
    global vectorstore

    if vectorstore is None:
        print("  [Function Error] LangChain vector store is not initialized.")

        return {"status": "error", "message": "Vector store not ready.", "context": ""}


    try:

        print(f"  Performing similarity search (k=3)...")

        retrieved_docs = vectorstore.similarity_search(query, k=3)

        if not retrieved_docs:
            print("  [Function Info] No relevant documents found.")
            return {"status": "success", "message": "No relevant context found.", "context": ""}
        else:

            context_list = []
            print(f"  [Function Info] Retrieved {len(retrieved_docs)} chunks.")
            for i, doc in enumerate(retrieved_docs):

                context_chunk = (
                    f"--- Context Chunk {i+1} ---\n"
                    f"Post ID: {doc.metadata.get('post_id', 'N/A')}\n"

                    f"Content: {doc.page_content}"
                )
                context_list.append(context_chunk)

            context_string = "\n\n".join(context_list)

            print(f"  [Function Success] Formatted context from {len(retrieved_docs)} chunks.")

            return {"status": "success", "message": "Context retrieved.", "context": context_string}

    except Exception as e:
        print(f"  [Function Error] Error during LangChain retrieval: {e}")
        traceback.print_exc()
        return {"status": "error", "message": f"An error occurred during retrieval: {str(e)}", "context": ""}


if callable(retrieve_relevant_context_langchain):
    print("\nRetrieval function 'retrieve_relevant_context_langchain' is defined.")
else:
     print("\nError: 'retrieve_relevant_context_langchain' function not defined correctly.")


Retrieval function 'retrieve_relevant_context_langchain' is defined.


In [35]:
# Define the model name
ADK_MODEL_NAME = 'gemini-2.0-flash'


# Agent instructions - updated to reflect function name change if needed and context format
rag_agent_instruction = """You are a factual Q&A assistant answering questions about Reddit posts. Your task is to answer user questions based *only* on the context provided by the 'retrieve_relevant_context_langchain' function. Follow these steps precisely:
1.  When you receive a user query, immediately call the 'retrieve_relevant_context_langchain' function with the exact user query to find relevant information from the indexed Reddit posts.
2.  Examine the result from the function. The 'context' field in the returned dictionary may contain multiple chunks of text identified by '--- Context Chunk X ---' and including metadata like 'Post ID'.
3.  If the function returns context (the 'context' field is not empty and status is 'success'), synthesize an answer based *solely* on the information present in ALL the provided context chunks. Start your response with: "Based on the provided information: ". Do not add any information not present in the context. You can reference Post IDs if helpful.
4.  If the function returns a message indicating no relevant context was found (e.g., the 'context' field is empty or status is not 'success'), respond *exactly* with: "I could not find information about that in the indexed Reddit posts."
5.  Do not use your general knowledge or any information outside the explicitly provided context from the function. If the function returns an error status, state that you encountered an issue retrieving information.
"""

rag_agent = None
vectorstore_ready = 'vectorstore' in globals() and vectorstore is not None
if os.environ.get('GOOGLE_API_KEY') and vectorstore_ready:
    try:
        rag_agent = Agent(
            name="hybrid_rag_agent_v2",
            model=ADK_MODEL_NAME,
            # Agent description
            description="Answers questions using RAG based on Reddit posts, calling a retrieval function that uses LangChain internally.",
            instruction=rag_agent_instruction,
            # Pass the raw function object in the list
            tools=[retrieve_relevant_context_langchain],
        )
        print(f"ADK Agent '{rag_agent.name}' created successfully with model '{ADK_MODEL_NAME}'.")
        print(f"Agent Tools: {[tool.__name__ for tool in rag_agent.tools]}") # Verify tool registration
    except Exception as e:
        print(f"ERROR creating ADK Agent (Model: {ADK_MODEL_NAME}): {e}")
        traceback.print_exc()
else:
    if not os.environ.get('GOOGLE_API_KEY'):
        print("Agent creation skipped: Google API Key is not configured.")
    elif not vectorstore_ready:
        print("Agent creation skipped: LangChain Vector Store is not ready (check Cell 5).")

ADK Agent 'hybrid_rag_agent_v2' created successfully with model 'gemini-2.0-flash'.
Agent Tools: ['retrieve_relevant_context_langchain']


In [44]:
adk_runner = None
adk_session_service = None

if rag_agent:
    adk_session_service = InMemorySessionService()
    adk_runner = Runner(
        agent=rag_agent,
        app_name="SimpleRAGTestApp",
        session_service=adk_session_service,
    )
    print("ADK Runner and Session Service initialized.")
else:
    print("ADK Runner initialization skipped because the Agent was not created.")

ADK Runner and Session Service initialized.


In [45]:
async def ask_rag_agent(query: str):
    """
    Sends a query to the configured RAG agent, verifies session creation,
    and prints the interaction flow and final response.
    Returns the final text response from the agent.
    """
    if not adk_runner or not adk_session_service:
        print("ERROR: ADK Runner or Session Service is not initialized. Cannot run query.")
        return "Error: ADK components not ready."

    session_id = f"rag_test_session_{os.urandom(4).hex()}"
    user_id = "colab_user_test" # Consistent user ID
    # Get app_name from the runner object
    app_name = adk_runner.app_name if adk_runner else "SimpleRAGTestApp" # Fallback name if needed

    print(f"\n=================================================")
    print(f"--- Starting Query for: '{query}' (Session: {session_id}) ---")
    print(f"--- App: {app_name}, User: {user_id} ---")
    print(f"=================================================")

    final_response = "Agent execution did not yield a final text response."
    session_verified = False # Flag to track if session seems okay before running

    try:
        # --- Step 1: Attempt to Create Session ---
        print(f"  Attempting to create session: {session_id}")
        adk_session_service.create_session(
            app_name=app_name,
            user_id=user_id,
            session_id=session_id
        )
        print(f"  ADK 'create_session' called for {session_id} (No immediate error thrown).")

        # --- Step 2: Attempt to Verify Session Existence (Corrected Call) ---
        try:
            print(f"  Verifying session {session_id} using get_session...")
            # *** CORRECTED CALL: Added app_name and user_id ***
            retrieved_session = adk_session_service.get_session(
                app_name=app_name,
                user_id=user_id,
                session_id=session_id
            )
            # Check if the retrieved session object is valid (might be specific to ADK version)
            # For InMemorySessionService, it likely returns the session state dict or raises error
            if retrieved_session is not None: # Basic check
                 print(f"  VERIFICATION SUCCESS: Successfully retrieved session {session_id} using get_session().")
                 session_verified = True
            else:
                 print(f"  VERIFICATION WARNING: get_session({session_id}) returned None.")
        except Exception as get_e:
            print(f"  VERIFICATION ERROR: Failed to retrieve session {session_id} using get_session(): {get_e}")
            # If get_session fails, the runner likely won't find it either.

        # --- Step 3: Run Agent (Only if Session Verification Succeeded) ---
        if session_verified:
            print(f"  Proceeding to call adk_runner.run_async for verified session {session_id}...")
            content = adk_types.Content(role='user', parts=[adk_types.Part(text=query)])

            async for event in adk_runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
                # ... (event processing loop - unchanged) ...
                event_type = type(event).__name__
                print(f"    [ADK Event]: {event_type}")

                if hasattr(event, 'tool_call') and event.tool_call:
                    tool_name = getattr(event.tool_call, 'name', 'N/A')
                    tool_args = getattr(event.tool_call, 'args', {})
                    print(f"      -> Agent requesting tool: '{tool_name}'")
                    print(f"         Args: {tool_args}")

                if hasattr(event, 'content') and event.content:
                    content_role = getattr(event.content, 'role', 'N/A')
                    if hasattr(event.content, 'parts'):
                        for i, part in enumerate(event.content.parts):
                            if hasattr(part, 'function_response') and part.function_response:
                                fr_name = getattr(part.function_response, 'name', 'N/A')
                                fr_resp = getattr(part.function_response, 'response', {})
                                print(f"      -> Function Response from '{fr_name}' received.")
                            if hasattr(part, 'text') and part.text:
                                if content_role == 'model':
                                     print(f"      -> Model Text Response Part: '{part.text[:150]}{'...' if len(part.text)>150 else ''}'")
                                     final_response = part.text

            print(f"\n--- Agent execution loop finished for Session: {session_id} ---")

        else:
             print(f"--- Agent execution skipped because session {session_id} could not be verified ---")
             final_response = "Error: Failed to create or verify ADK session."

    except Exception as e:
        print(f"ERROR during agent execution/session handling for {session_id}: {e}")
        if "Session not found" in str(e) and session_verified:
             print("  >>> CRITICAL: Got 'Session not found' during run_async even after verification passed!")
        traceback.print_exc()
        final_response = f"An error occurred: {str(e)}"

    finally:
        # --- Step 4: Clean up the session ---
        try:
            if adk_session_service and adk_runner:
                print(f"  Attempting to delete session {session_id}...")
                adk_session_service.delete_session(
                    app_name=app_name, # Use correct app_name
                    user_id=user_id,
                    session_id=session_id
                )
                print(f"  Session {session_id} delete requested.")
        except Exception as del_e:
            print(f"  Info/Warning during session deletion for {session_id}: {del_e}")

    print(f"\n>>> Final Response: {final_response}")
    print(f"=================================================")
    return final_response

In [48]:
# Define an async function to run the tests
async def run_all_tests():
    if not adk_runner:
        print("Cannot run tests - ADK Runner is not initialized.")
        return

    # Test Case 1: Query likely in KB
    await ask_rag_agent("What are the stocks that are being talked about?")

    await ask_rag_agent("Which stalks seem to have the most positive sentiment in the entire post list?")



# --- IMPORTANT ---
# In Colab/Jupyter, you run the async function like this:
await run_all_tests()


--- Starting Query for: 'What are the stocks that are being talked about?' (Session: rag_test_session_165a5218) ---
--- App: SimpleRAGTestApp, User: colab_user_test ---
  Attempting to create session: rag_test_session_165a5218
  ADK 'create_session' called for rag_test_session_165a5218 (No immediate error thrown).
  Verifying session rag_test_session_165a5218 using get_session...
  VERIFICATION SUCCESS: Successfully retrieved session rag_test_session_165a5218 using get_session().
  Proceeding to call adk_runner.run_async for verified session rag_test_session_165a5218...


    [ADK Event]: Event

--- LangChain Retrieval Function called with query: 'What are the stocks that are being talked about?' ---
  Performing similarity search (k=3)...
  [Function Info] Retrieved 3 chunks.
  [Function Success] Formatted context from 3 chunks.
    [ADK Event]: Event
      -> Function Response from 'retrieve_relevant_context_langchain' received.
    [ADK Event]: Event
      -> Model Text Response Part: 'Based on the provided information: The stocks being talked about are TSLA (Tesla), domestic lithium refinery stocks, NOW, and GOOG. Post ID 1k3ou80 di...'

--- Agent execution loop finished for Session: rag_test_session_165a5218 ---
  Attempting to delete session rag_test_session_165a5218...
  Session rag_test_session_165a5218 delete requested.

>>> Final Response: Based on the provided information: The stocks being talked about are TSLA (Tesla), domestic lithium refinery stocks, NOW, and GOOG. Post ID 1k3ou80 discusses Tesla earnings. Post ID 1k57wvu discusses domesti

    [ADK Event]: Event

--- LangChain Retrieval Function called with query: 'Which stalks seem to have the most positive sentiment in the entire post list?' ---
  Performing similarity search (k=3)...
  [Function Info] Retrieved 3 chunks.
  [Function Success] Formatted context from 3 chunks.
    [ADK Event]: Event
      -> Function Response from 'retrieve_relevant_context_langchain' received.
    [ADK Event]: Event
      -> Model Text Response Part: 'Based on the provided information: Post ID 1k4jh6e mentions a $3.8k profit on NFE puts. Post ID 1k7r9id refers to a $23k gain on GOOG. Post ID 1k5znh7...'

--- Agent execution loop finished for Session: rag_test_session_84ce2342 ---
  Attempting to delete session rag_test_session_84ce2342...
  Session rag_test_session_84ce2342 delete requested.

>>> Final Response: Based on the provided information: Post ID 1k4jh6e mentions a $3.8k profit on NFE puts. Post ID 1k7r9id refers to a $23k gain on GOOG. Post ID 1k5znh7 discusses hedging gambles 